In [173]:
using LinearAlgebra # do operacji macierzowych i testowania
using RandomNumbers.MersenneTwisters # do MT19937()
using Printf
using DelimitedFiles

epsilon = 10e-8;

In [181]:
# test data
function Ab_read_from_file(path)    
    local D1 = (open(readdlm, path))
    return M = (A=D1[:, 1:end-1], b=D1[:, end])
end

function randEchelonGen(n, val_range)
    local M = rand(val_range, n, n)
    for i in 1:length(M[:, 1])
        M[i, 1:i-1] = zeros(i-1)'
    end
    return (A=M, b=rand(val_range, n))
end

begin
    local Id_100x100          = (b=rand(-10.0:10.0, 100), A=Matrix{Float64}(I, 100, 100))
    local small_3x3           = (A=[1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0], b=[10.0 12.0 13.0]')
    local big01_100x100       = Ab_read_from_file("gaussMatrix1.txt")
    local big02_100x100       = Ab_read_from_file("gaussMatrix2.txt")
    local big03_100x100       = Ab_read_from_file("gaussMatrix3.txt")
    local rand_echelon100x100 = randEchelonGen(100, -100.0:100.0)

    # tests consist of fieds M=matrix, desc=description of test and exact=wanted
    tests = [
        (M=Id_100x100,    desc="Identity 100x100 matrix with random b")
        (M=small_3x3 ,    desc="Small hardcoded 3x3 test")
        (M=big01_100x100, desc="Big 100x100 hardcoded test [1]")
        (M=big02_100x100, desc="Big 100x100 hardcoded test [2]")
        (M=big03_100x100, desc="Big 100x100 hardcoded test [3]")
        (M=big03_100x100, desc="Random echelon form 100x100 (might crash)")
    ];
end;

In [182]:
# test na macierzach. 
# f: funkcja zwracającą wynik do przetestowania z testem
# name: nazwa metody liczącej rząd macierzy
# rng: generator liczb losowych. Można przekazać np. MT19937(<seed>) aby generować te same zestawy losowych danych
# exact_f: funkcja do liczenia dokładnego wyniku, chyba że test zawiera pole `exact`, wtedy to ono jest użyte
# equal_f: opcjonalne do porównywania wartości wyliczonej przez f() i exact_f() lub test.exact_f(). Domyślnie: `==`
function test(f, f_name, exact_f; rng=nothing, equal_f=(x, y)->x==y)
    local _rand = rng==nothing ? rand : (args...)->rand(rng, args...) # custom rand if generator is set
    
    @printf(">>> Random test ID: [\e[1;35m%.16f\e[m]\n", _rand())

    function _do_test(test)
        local t1 = @elapsed exact = haskey(test, :exact_f) ? test.exact_f(test.M) : exact_f(test.M)
        local t2 = @elapsed computed = f(test.M)
        local success = equal_f(exact, computed)
         @printf("[%5s] [exact:\ttime:%f] [%s:\ttime:%f] <<< %s\n", 
                 success ? "\e[1;32mPASS\e[m" : "\e[1;31mFAIL\e[m", t1, f_name, t2, test.desc)
    end

    @printf("> Hardcoded tests:\n")
    for test in tests
        _do_test(test)
    end
    
    flush(stdout)
    @printf(">>> TESTS END\n")
end

function getPrecisionU(T=Float64) return 0.5*2.0^-precision(T) end
# zamienia liczby zmiennoprzecinkowe w macierzy na zera jeśli |x|≦|maxError|
function filterZeros!(matrix, maxError=epsilon) map!(x->abs(x)<=abs(maxError) ? 0.0 : x, matrix) end
;

In [183]:
# solves Ax=b where Aₙₓₙ matrix of coefficients, bₙₓ₁ matrix of results for linear equations
function solve!(A, b)
    local M = [A b]'
    local n = length(M[1, :])
    local isZero = x->abs(x)<=epsilon
    # Gauss reduce on columns
    for col in 1:n
        # swap-in column with nonzero value
        local allzero = true
        for j in col:n 
            if (!isZero(M[col, j])) 
                M[:, col], M[:, j], allzero = M[:, j], M[:, col], false
                break
            end
        end
        if (allzero) continue end # skip to next step iff row with zeros till the end
        
        # gauss reduce columns
        for next_col in col+1:n
            M[:, next_col] -= M[:, col]*M[col, next_col]/M[col, col] # denom !=0 cuz we swapped
        end
    end
    
    for col in n:-1:1
        if (isZero(M[col, col]))
            if (!isZero(M[end, col])) return NaN*ones(length(b)) end # no solutions fuz we have 0*x = y (!=0)
            return Inf*ones(length(b)) # inf solutions cuz we have 0*x = 0
        end
        M[:, col] /= M[col, col] # calculate b[col]
        for j in col-1:-1:1
            M[:, j] -= M[col, j]*M[:, col]
        end
    end
    return M[end, :]
    
end
;

In [200]:
test(M->solve!(M.A, M.b), "solve!", M->\(M.A, M.b); equal_f=(v,w)->abs(norm(v-w))<=epsilon, rng=MT19937())

>>> Random test ID: [0.3222212102264166]
> Hardcoded tests:
[PASS] [exact:	time:0.000020] [solve!:	time:0.008423] <<< Identity 100x100 matrix with random b
[PASS] [exact:	time:0.000015] [solve!:	time:0.000008] <<< Small hardcoded 3x3 test
[PASS] [exact:	time:0.007019] [solve!:	time:0.007881] <<< Big 100x100 hardcoded test [1]
[PASS] [exact:	time:0.008467] [solve!:	time:0.008236] <<< Big 100x100 hardcoded test [2]
[PASS] [exact:	time:0.004474] [solve!:	time:0.007215] <<< Big 100x100 hardcoded test [3]
[PASS] [exact:	time:0.003262] [solve!:	time:0.008310] <<< Random echelon form 100x100 (might crash)
>>> TESTS END
